In [2]:
from BallHeaterDriver import BallHeaterDriver
import time
import pandas as pd
import matplotlib.pyplot as plt 
from list_ports import list_ports

In [3]:
list_ports()

[{'port': '/dev/cu.Bluetooth-Incoming-Port',
  'manufacturer': None,
  'serial': None},
 {'port': '/dev/cu.MEGABOOM3', 'manufacturer': None, 'serial': None},
 {'port': '/dev/cu.usbserial-210',
  'manufacturer': 'Silicon Labs',
  'serial': '4006a22f2575ec11bcf72b71c00adeb6'}]

In [5]:
ball_heater = BallHeaterDriver(port = '/dev/cu.usbserial-210')

In [29]:
# ball_heater.send_command('set_control_mode', [3])
ball_heater.send_command('set_heater_pwm_manual', [10])
ball_heater.send_command('status')

(True,
 {'target_temp': 14.5,
  'ball_heater_pwm': 10.0,
  'heater_temp': 22.310577392578125,
  'aux_therm_temp': 25.8128662109375,
  'control_mode': 3})

In [30]:
ball_heater.send_command('return_pid_parameters')

(True, b'\x00\x00 A\x00\x00\x00@\xcd\xcc\xcc=')

In [26]:
ball_heater.send_command('set_target_temp', [10])
ball_heater.send_command('status')

(True,
 {'target_temp': 10.0,
  'ball_heater_pwm': 50.0,
  'heater_temp': 22.259033203125,
  'aux_therm_temp': 25.74273681640625,
  'control_mode': 1})